In [1]:
%load_ext autoreload
%autoreload 2
import serial, time, io, datetime
from serial import Serial
# import sys
import time
import pdb
import csv
from IPython.core.debugger import set_trace
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time

## Sending data to google sheet V01

In [ ]:
# Google Sheet API

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('Trail-a7cf89fbdcf3.json', scope)
print(credentials)
# Import credentials
client = gspread.authorize(credentials)

# Open sheet
sheet = client.open("IOT").worksheets()

Light=[]
Humidity=[]
DHT_Temp=[]
Soil=[]
Thermo=[]
ser = serial.Serial('COM4', 9600)
while ser.isOpen():
        datastring = ser.readline().decode()
        split = datastring.split(",")
        Light.append(split[0])
        Humidity.append(split[1])
        DHT_Temp.append(split[2])
        Soil.append(split[3])
        Thermo.append(split[4][:-2])
        for i in sheet:
            curr_time = time.localtime()
            time_str = time.strftime("%m/%d/%Y %H:%M:%S",curr_time)
            # Send data 
            i.append_row([time_str, Light[len(Light)-1], Humidity[len(Humidity)-1], DHT_Temp[len(DHT_Temp)-1], Soil[len(Soil)-1], Thermo[len(Thermo)-1]])

## Sending data to google sheet V02

In [2]:
# Function to segregate data
def sensor_group(datastring):
    if datastring[0] == 'A':
        Light.append(datastring[1:len(datastring)][:-2])
    else:
        if datastring[0] == 'B':
            Humidity.append(datastring[1:len(datastring)][:-2])
        else:
            if datastring[0] == 'C':
                DHT_Temp.append(datastring[1:len(datastring)][:-2])     
            else:
                if datastring[0] == 'D':
                    Soil.append(datastring[1:len(datastring)][:-2])
                else:
                    if datastring[0] == 'E':
                        Thermo.append(datastring[1:len(datastring)][:-2])
                    
    return Light, Humidity, DHT_Temp, Soil, Thermo

# Google Sheet API
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('Trail-a7cf89fbdcf3.json', scope)
# Import credentials
client = gspread.authorize(credentials)
# Open sheet
sheet = client.open("IOT").worksheets()
#----------------------------------------------------------
# Start Serial Communication
ser = serial.Serial('COM4', 9600)
sio = io.TextIOWrapper(
   io.BufferedRWPair(ser, ser, 1), 
   encoding= 'ascii', newline='\r')
Light = []
Humidity=[]
DHT_Temp=[]
Soil=[]
Thermo=[]

while ser.isOpen():
#     try:
        datastring = ser.readline().decode()
        # Call function
        Light, Humidity, DHT_Temp, Soil, Thermo = sensor_group(datastring) 
        for i in sheet:
            curr_time = time.localtime()
            time_str = time.strftime("%m/%d/%Y %H:%M:%S",curr_time)
            # Clean empty array
            if Humidity == []:
                Humidity.append('0')
            if DHT_Temp == []:
                DHT_Temp.append('0')
            if Soil == []:
                Soil.append('0')
            if Thermo == []:
                Thermo.append('0')
            # Send data 
            i.append_row([time_str, Light[len(Light)-1], Humidity[len(Humidity)-1], DHT_Temp[len(DHT_Temp)-1], Soil[len(Soil)-1], Thermo[len(Thermo)-1]])
#     except:
#         print("Something is wrong")
#         break        

SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))

# Write data in text file

In [ ]:
ser = serial.Serial('COM4', 9600)
sio = io.TextIOWrapper(
   io.BufferedRWPair(ser, ser, 1), 
   encoding= 'ascii', newline='\r')

filename="Rawdata01.txt"    
with open(filename, 'a') as f:
    while ser.isOpen():
        try:
            datastring = ser.readline().decode()
            f.write(datastring)
            f.flush()
        except:
            print("Keyboard Interrupt")
            break    
            
filename.close()
ser.close()   

## Storing Data Localy

In [ ]:
def sensor_group(datastring):
    if datastring[0] == 'A':
        Humidity.append(datastring[1:6][:-1])
#         print(Humidity)
    else:
        if datastring[0] == 'B':
            DHT_Temp.append(datastring[1:6][:-1])
    #                 print(DHT_Temp)
        else:
            if datastring[0] == 'C':
                Soil.append(datastring[1:len(datastring)][:-2])
    #                     print(Soil)       
            else:
                if datastring[0] == 'D':
                    Thermo.append(datastring[1:len(datastring)][:-2])
    #                         print(Thermo)  
    return Humidity, DHT_Temp, Soil,Thermo

ser = serial.Serial('COM4', 9600)
sio = io.TextIOWrapper(
   io.BufferedRWPair(ser, ser, 1), 
   encoding= 'ascii', newline='\r')
Humidity=[]
DHT_Temp=[]
Soil=[]
Thermo=[]
# filename="Rawdata01.txt"    
# with open(filename, 'a') as f:
while ser.isOpen():
#     try:
        datastring = ser.readline().decode()
        Humidity, DHT_Temp, Soil,Thermo = sensor_group(datastring)

        writer = pd.ExcelWriter('SignalProfile01.xlsx', engine='xlsxwriter')
        df = pd.DataFrame.from_dict({'Humidity':Humidity, 'DHT_Temp': DHT_Temp,'Soil': Soil, 'Thermo': Thermo}, orient='index')
        df.to_excel(writer, sheet_name='Sheet1')

#             f.write(datastring)
#             f.flush()
#     except:
#         print("Keyboard Interrupt")
#         break    
# print(A)            
# filename.close()
# ser.close()   

In [ ]:
ser = serial.Serial('COM4', 9600)
while True:
    data = ser.readline()[:-2]
    print(data)

In [ ]:
ser = serial.Serial('COM4', 9600)
while True:
    data = ser.read()#[:-2]
    print(data)

In [ ]:
from __future__ import print_function
import serial, time, io, datetime
from serial import Serial

addr = "COM4" ## serial port to read data from
baud = 9600 ## baud rate for instrument

ser = serial.Serial(
    port = addr,\
    baudrate = baud,\
    parity=serial.PARITY_NONE,\
    stopbits=serial.STOPBITS_ONE,\
    bytesize=serial.EIGHTBITS,\
    timeout=0)


print("Connected to: " + ser.portstr)

# filename="myFile.txt"
# datafile=open(filename, 'a')

# ## this will store each line of data
# seq = []
# count = 1 ## row index

# while True:
#     for i in ser.readline()[:-2]:
#         seq.append(i) ## convert from ACSII?
# #         joined_seq = ''.join(str(v) for v in seq) ## Make a string from array
#         print(seq)

#         if i == '\n':
#             datafile.write("Line: " + str(count) + "" + str(datetime.datetime.now()) + joined_seq) ## append a timestamp to each row of data
#             seq = []
#             count += 1
#             break
# datafile.close()
# ser.close()

In [ ]:
ser = serial.Serial('COM4', 9600)
seq = []
while True:
     for i in ser.readline()[:-2]:
#         seq.append(i) ## convert from ACSII?
#         joined_seq = ''.join(str(v) for v in seq) ## Make a string from array
        print(i)

In [ ]:
# with serial.Serial('COM4', 9600) as ser:
ser = serial.Serial('COM4', 9600)
#     x = ser.read(80) 
#     print(x)
line = ser.readline()[:-2]
print(line)
# flush any junk left in the serial buffer
# ser.flushInput()
# # ser.reset_input_buffer() # for pyserial 3.0+
# run = True

In [ ]:
start_time = time()
timepoints = []
ydata = []
while True:
    data = ser.readline()[:-2] #the last bit gets rid of the new-line chars
        # sometimes the incoming data is garbage, so just 'try' to do this
        # store the entire dataset for later
    ydata.append(data)
    timepoints.append(time()-start_time)
    current_time = timepoints[-1]
    print(data.type)


In [ ]:
print(ydata[0])

In [ ]:
%matplotlib notebook

# run this in a Jupyter (IPython) Notebook!
# modified from http://www.lebsanft.org/?p=48
# http://pyserial.readthedocs.org/en/latest/pyserial_api.html
import serial
import numpy as np
from matplotlib import pyplot as plt
from time import time

# If you're not using Linux, you'll need to change this
# check the Arduino IDE to see what serial port it's attached to
ser = serial.Serial('COM4', 9600)

# set plot to animated
plt.ion() 

start_time = time()
timepoints = []
ydata = []
yrange = [-0.1,5.1]
view_time = 4 # seconds of data to view at once
duration = 24 # total seconds to collect data

fig1 = plt.figure()
# http://matplotlib.org/users/text_props.html
fig1.suptitle('live updated data', fontsize='18', fontweight='bold')
plt.xlabel('time, seconds', fontsize='14', fontstyle='italic')
plt.ylabel('potential, volts', fontsize='14', fontstyle='italic')
plt.axes().grid(True)
line1, = plt.plot(ydata,marker='o',markersize=4,linestyle='none',markerfacecolor='red')
plt.ylim(yrange)
plt.xlim([0,view_time])

# flush any junk left in the serial buffer
ser.flushInput()
# ser.reset_input_buffer() # for pyserial 3.0+
run = True

# collect the data and plot a moving frame
while run:
    ser.reset_input_buffer()
    data = ser.readline()[:-2]
    
    # sometimes the incoming data is garbage, so just 'try' to do this
    try:
        # store the entire dataset for later
        ydata.append(float(data[0])*5.0/1024)
        timepoints.append(time()-start_time)
        current_time = timepoints[-1]
        
        # update the plotted data
        line1.set_xdata(timepoints)
        line1.set_ydata(ydata)
        
        # slide the viewing frame along
        if current_time > view_time:
            plt.xlim([current_time-view_time,current_time])
            
        # when time's up, kill the collect+plot loop
        if timepoints[-1] > duration: run=False
    
    # if the try statement throws an error, just do nothing
    except: pass
    
    # update the plot
    fig1.canvas.draw()

# plot all of the data you collected
fig2 = plt.figure()
# http://matplotlib.org/users/text_props.html
fig2.suptitle('complete data trace', fontsize='18', fontweight='bold')
plt.xlabel('time, seconds', fontsize='14', fontstyle='italic')
plt.ylabel('potential, volts', fontsize='14', fontstyle='italic')
plt.axes().grid(True)

plt.plot(timepoints, ydata,marker='o',markersize=4,linestyle='none',markerfacecolor='red')
plt.ylim(yrange)
fig2.show()

ser.close()